<a href="https://colab.research.google.com/github/pitfisher/miem_face_recognition/blob/main/FR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepface
!pip install matplotlib

In [ ]:
import cv2
import time
import os
import glob
from PIL import Image
from deepface import DeepFace
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
drive.mount('/content/gdrive')

input_folder = ''
tmp_folder = "tmp"
os.makedirs(tmp_folder, exist_ok=True)

resolutions = [1024, 768, 512, 256, 224, 128, 112, 96, 64, 32]

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def crop_to_square(image, face_info):
    x, y, w, h = face_info['facial_area']['x'], face_info['facial_area']['y'], face_info['facial_area']['w'], \
                 face_info['facial_area']['h']

    outer_border_scale = 0.5
    img_w, img_h, _ = image.shape

    square_size = int(max(w, h) / outer_border_scale)
    to_closest_border = min(x, y, img_w - x, img_h - y)

    square_size = min(square_size, to_closest_border * 2)

    new_x = x + w // 2 - square_size // 2
    new_y = y + h // 2 - square_size // 2

    cropped_img = image[new_y: new_y + square_size, new_x: new_x + square_size]

    return cropped_img

In [ ]:
def resize_image(image, new_resolution):
    resized_image = cv2.resize(image, new_resolution)
    return resized_image

In [ ]:
def save_to_csv(data, columns, filename):
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(filename + '.csv', index=False)

In [ ]:
files = glob.glob(os.path.join(input_folder, "**", "*.JPG"), recursive=True)

data = {}

for img_path in tqdm(files, desc="Processing images", unit="image"):
    path = img_path.split(os.path.sep)
    filename = f'{path[-2]}_{path[-1]}'
    ds_name = f'VGG_{path[-4]}_{path[-3]}'

    try:
        face_info = DeepFace.extract_faces(img_path=img_path, detector_backend="retinaface")[0]
    except ValueError as e:
     #   print(f"No face found for {filename}, error message: {str(e)}")
        continue

    image = cv2.imread(img_path)

    cropped_img = crop_to_square(image, face_info)
    resized_img = resize_image(cropped_img, (resolutions[0], resolutions[0]))

    metrics = []

    for resolution in resolutions:
        try:
            resized_img = resize_image(cropped_img, (resolution, resolution))

            start_time = time.perf_counter()

            result = DeepFace.verify(img1_path=cropped_img,
                                     img2_path=resized_img,
                                     enforce_detection=True,
                                     detector_backend="retinaface",
                                     model_name="VGG-Face",
                                     distance_metric="euclidean_l2"
                                     )

            end_time = time.perf_counter()
            fin_time = end_time - start_time

           # print(f'verify for {filename} in {ds_name} for resolution {resolution} processed in {fin_time}')
            metrics.append(f'{result["distance"]:.5f}')
            metrics.append(f'{fin_time:.5f}')

        except ValueError as e:
           # print(f"No face found for {filename} and resolution {resolution}, error message: {str(e)}")
            metrics.extend([None, None])

    if ds_name not in data:
        data[ds_name] = []

    data[ds_name].append([filename] + metrics)

columns = ['name'] + [f"{res}_{metric}" for res in resolutions for metric in ["distance", "time"]]

for ds_name, ds in data.items():
    save_to_csv(ds, columns, ds_name)

Processing images: 100%|██████████| 9/9 [00:35<00:00,  3.96s/image]
